In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium

In [2]:
# Load Russian vaccination data (July 2021)
df=pd.read_csv('covidrusstates.csv')

In [3]:
# Clean data
df=df[['name','vaccinated','ratio']]
df['vaccinated']=df['vaccinated'].apply(lambda x: x.replace(',',''))
df['vaccinated']=df['vaccinated'].apply(lambda x: int(x.replace(' ','')))
df['ratio']=df['ratio'].apply(lambda x: float(x))
df.sort_values('name', ascending=True, inplace=True)

In [4]:
# Load Russian Federal Subjects shapefile
russia=gpd.read_file('admin_level_4')

In [5]:
# Clean data
russia=russia[['name_en','geometry']]
russia.sort_values('name_en', ascending=True, inplace=True)

In [6]:
# Merge maps
mergedmap=russia.merge(df, left_on='name_en', right_on='name')

In [7]:
# Optimize projection to Russia
mergedmap=mergedmap.to_crs(epsg=3576)

In [8]:
# Load base map
basemap = folium.Map(location=(50, 30.428932), zoom_start=2, tiles='stamenwatercolor')

In [9]:
# Apply data on basemap
folium.Choropleth(geo_data=mergedmap, data=mergedmap, key_on='feature.properties.name_en',
                  columns=['name_en', 'ratio'], fill_color='Greens', fill_opacity=1,
                  legend_name='Rate of vaccination (%)').add_to(basemap)

In [10]:
# Save the map as .html file
basemap.save('rusvaccstatus.html')